In [37]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [94]:
def print_cross_validation_accuracy(model, x, y, cross_val):
    scores = cross_val_score(model, x, y, cv = cross_val)
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [105]:
products_data = pandas.DataFrame(pandas.read_csv("products.csv", sep = ','))
products_data = products_data[products_data.groupby('label_category')['label_category'].transform('count').ge(8)]
products_label = products_data[['label_category']].values.ravel()
products_data_text=[]
for index, row in products_data.iterrows():
    products_data_text.append(                         
                          str(row['name']) + 
                          str(row['brand']) + 
                          str(row['location']) + 
                          str(row['base_category']) +               
                          str(row['top_description']) + 
                          str(row['feature_description']))

In [106]:
X_text = TfidfVectorizer().fit_transform(products_data_text).toarray()
y_text = products_label
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(X_text, y_text, test_size=0.33, random_state=1511849)

In [108]:
#knn_clf = KNeighborsClassifier(n_neighbors=7)

text_clf = Pipeline([('clf', LinearSVC())])
text_clf.fit(X_text_train, y_text_train)

print_cross_validation_accuracy(model = text_clf, x = X_text, y = y_text, cross_val = 10)

c:\users\nguyenhoangloc\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


[0.8442623  0.85833333 0.88596491 0.87272727 0.91509434 0.86666667
 0.9009901  0.8989899  0.84375    0.80434783]
Accuracy: 0.87 (+/- 0.06)
